Obliczenie statystyk

In [ ]:
import os
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.features import geometry_mask
from rasterio.transform import from_origin


output_folder_path = ''
folder_path = ''
file_name = ''
shp_files = ['']
def calculate_statistics(raster, transform, shp_file_path, region_name):
    boundaries = gpd.read_file(shp_file_path)

    raster_mask = geometry_mask(boundaries.geometry, transform=transform, out_shape=raster.shape)
    raster_masked = np.where(raster_mask, np.nan, raster)

    unique_values = np.unique(raster_masked[~np.isnan(raster_masked)])
    print(f'Unique classes for {region_name}:', unique_values)

    stats = {}

    total_area_pixels = np.sum(~np.isnan(raster_masked))
    for i in unique_values:
        class_pixels = np.sum(raster_masked == i)
        class_area_m2 = class_pixels * transform.a * -transform.e  
        class_area_km2 = class_area_m2 / 1e6 
        class_percentage = (class_pixels / total_area_pixels) * 100  

        stats[f'Class {int(i)}'] = {
            'Total area [m²]': class_area_m2,
            'Total area [km²]': class_area_km2,
            'Percentage share': class_percentage,
        }

    return stats

with rasterio.open(os.path.join(folder_path, file_name)) as src:
    raster = src.read(1) 
    transform = src.transform

    for shp_file_path in shp_files:
        region_name = os.path.splitext(os.path.basename(shp_file_path))[0]
        
        stats = calculate_statistics(raster, transform, shp_file_path, region_name)

        output_file_path = os.path.join(output_folder_path, f'statistics_{file_name}.txt') 
        with open(output_file_path, 'w', encoding='utf-8') as f:
            f.write(f'Statistics for region: {region_name}\n')
            for class_name, class_stats in stats.items():
                f.write(f'\nStatistics for {class_name}:\n')
                for stat_name, value in class_stats.items():
                    f.write(f'{stat_name}: {value:.2f}\n')

Obliczenie map ciepła (heatmapy)

In [ ]:
import re
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os


file_path = ''
def load_f1_scores(file_path):
    with open(file_path, 'r') as file:
        text = file.read()

    pattern = r'Result for (AOI\d+_\d+_ts_predicted\.csv):.*?F1-score for class 6:\s+(\d\.\d+)' 
    matches = re.findall(pattern, text, re.DOTALL)
    data = {
        'filename': [match[0] for match in matches],
        'f1_class_6': [float(match[1]) for match in matches]
    }
    return pd.DataFrame(data)
df = load_f1_scores(file_path)
shp_folder = ''
result_gdf_list = []
for _, row in df.iterrows():
    match = re.search(r'AOI\d+_(\d+)_ts_predicted', row['filename'])
    if match:
        aoi_number = match.group(1)
        shp_filename = os.path.join(shp_folder, f"AOIOgun_{aoi_number}.shp")
        print(f"Trying to load: {shp_filename}")  
        if os.path.exists(shp_filename):
            gdf = gpd.read_file(shp_filename)
            gdf['f1_class_6'] = row['f1_class_6']
            result_gdf_list.append(gdf)
        else:
            print(f"File not found: {shp_filename}")
    else:
        print(f"Pattern not matched for filename: {row['filename']}")

if result_gdf_list:
    result_gdf = gpd.GeoDataFrame(pd.concat(result_gdf_list, ignore_index=True))
else:
    raise ValueError("No shapefiles found. Please check the file paths.")

contour_shp_path = ''
contour_gdf = gpd.read_file(contour_shp_path)
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
cmap = plt.cm.Greens
result_gdf.plot(column='f1_class_6', cmap=cmap, linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
contour_gdf.plot(ax=ax, edgecolor='gray', facecolor='none', linewidth=1.0)
for idx, row in result_gdf.iterrows():
    centroid = row['geometry'].centroid
    plt.annotate(f"{row['f1_class_6']*100:.1f}%", xy=(centroid.x, centroid.y),
                 horizontalalignment='center', fontsize=10, color='black')
plt.title('Heatmap of F1-scores for Class 6 - POL Index')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
import re
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os

folder_path = ''

def load_data(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    pattern = r'Statistics for Class 6:\s+Total area \[m²\]: \d+\.\d+\s+Total area \[km²\]: \d+\.\d+\s+Percentage share: (\d+\.\d+)'
    match = re.search(pattern, text)
    if match:
        return float(match.group(1))
    else:
        return None
data = []
for filename in os.listdir(folder_path):
    if filename.startswith("statistics_AOIOgun_") and filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        percentage_share = load_data(file_path)
        if percentage_share is not None:
            data.append({
                'AOI': filename.split("_")[-1].split(".")[0],
                'percentage_share': percentage_share
            })

df = pd.DataFrame(data)

shp_folder = ''
result_gdf_list = []

for _, row in df.iterrows():
    shp_filename = os.path.join(shp_folder, f"AOIOgun_{row['AOI']}.shp")
    print(f"Trying to load: {shp_filename}")  
    if os.path.exists(shp_filename):
        gdf = gpd.read_file(shp_filename)
        gdf['percentage_share'] = row['percentage_share']
        result_gdf_list.append(gdf)
    else:
        print(f"File not found: {shp_filename}")

if result_gdf_list:
    result_gdf = gpd.GeoDataFrame(pd.concat(result_gdf_list, ignore_index=True))
else:
    raise ValueError("No shapefiles found. Please check the file paths.")

contour_shp_path = ''
contour_gdf = gpd.read_file(contour_shp_path)
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
cmap = plt.cm.Greens
result_gdf.plot(column='percentage_share', cmap=cmap, linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
contour_gdf.plot(ax=ax, edgecolor='gray', facecolor='none', linewidth=1.0)
for idx, row in result_gdf.iterrows():
    centroid = row['geometry'].centroid
    plt.annotate(f"{row['percentage_share']:.2f}%", xy=(centroid.x, centroid.y),
                 horizontalalignment='center', fontsize=10, color='black')
plt.title('Heatmap gęstości lasów - DPSVIm')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()
